# Processing marker-gene data in QIIME2, part2

**Environment:** qiime2-2021.4

## How to use this notebook:
1. Activate the `qiime2-2021.4` conda environment.
    ```
   source $HOME/miniconda3/bin/activate # use the path in your local machine to activate miniconda
   conda activate qiime2-2021.4 # activate qiime2 conda environment
    ```
    
2. Launch Jupyter notebook:
    ```
   jupyter notebook
    ```  

In [ ]:
## Hide excessive warnings (optional):
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## change working directory to the project root directory
%cd ..

##  Import and visualize the filtered feature table

In [ ]:
# Import table
!qiime tools import \
  --input-path data/intermediate/filtering/table_merged_filtered.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV100Format \
  --output-path data/intermediate/qiime2/table_merged_filtered.qza

# Feature table summary 
!qiime feature-table summarize \
  --i-table data/intermediate/qiime2/table_merged_filtered.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/table_merged_filtered.qzv 

# Filtered taxonomic barplot 
!qiime taxa barplot \
  --i-table data/intermediate/qiime2/table_merged_filtered.qza \
  --i-taxonomy data/intermediate/qiime2/taxonomy_silva132.qza \
  --m-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/taxa_bar_plots_filtered.qzv

##  Phylogeny 

###  Filter representative sequences based on the filtered feature table

In [ ]:
!qiime feature-table filter-seqs \
  --i-data data/intermediate/qiime2/rep_seqs_merged.qza \
  --i-table data/intermediate/qiime2/table_merged_filtered.qza \
  --p-no-exclude-ids \
  --o-filtered-data data/intermediate/qiime2/rep_seqs_merged_filtered.qza

!qiime feature-table tabulate-seqs \
  --i-data data/intermediate/qiime2/rep_seqs_merged_filtered.qza \
  --o-visualization data/intermediate/qiime2/rep_seqs_merged_filtered.qzv

###  Reference-based fragment insertion with SEPP

In [ ]:
%%time
!qiime fragment-insertion sepp \
  --i-representative-sequences data/intermediate/qiime2/rep_seqs_merged_filtered.qza \
  --i-reference-database data/reference/sepp-refs-silva-128.qza \
  --o-tree data/intermediate/qiime2/insertion_tree.qza \
  --o-placements data/intermediate/qiime2/tree_placements.qza \
  --p-threads 16 \
  --p-debug

### Filter uninserted representative sequences from the feature table  

In [ ]:
!qiime fragment-insertion filter-features \
  --i-table data/intermediate/qiime2/table_merged_filtered.qza \
  --i-tree data/intermediate/qiime2/insertion_tree.qza \
  --o-filtered-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted.qza \
  --o-removed-table data/intermediate/qiime2/table_merged_filtered_sepp_uninserted.qza \
  --verbose

### Feature table summary

In [ ]:
!qiime feature-table summarize \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/table_merged_filtered_sepp_inserted.qzv 

!qiime feature-table summarize \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_uninserted.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/table_merged_filtered_sepp_uninserted.qzv 

## Alpha and beta diversity analysis: tilapia

### Filter samples

In [ ]:
!qiime feature-table filter-samples \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted.qza \
  --m-metadata-file data/metadata.tsv \
  --p-where "[Species]='Nile tilapia'" \
  --o-filtered-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_tilapia.qza

### Rarefaction analysis 

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_tilapia.qza \
  --i-phylogeny data/intermediate/qiime2/insertion_tree.qza \
  --p-max-depth 133398 \
  --p-steps 10 \
  --m-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/alpha_rarefaction_tilapia.qzv

### Generate core metric results

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny data/intermediate/qiime2/insertion_tree.qza \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_tilapia.qza \
  --m-metadata-file data/metadata.tsv \
  --p-sampling-depth 133398 \
  --output-dir data/intermediate/qiime2/core_metrics_results_tilapia

### Compare beta-diversity using robust Aitchison PCA 

In [ ]:
!qiime deicode rpca \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_tilapia.qza \
  --p-min-feature-count 10 \
  --p-min-sample-count 1000 \
  --output-dir data/intermediate/qiime2/rpca_tilapia

!qiime emperor biplot \
  --i-biplot data/intermediate/qiime2/rpca_tilapia/biplot.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --m-feature-metadata-file data/intermediate/qiime2/taxonomy_silva132.qza \
  --o-visualization data/intermediate/qiime2/rpca_tilapia/biplot.qzv \
  --p-number-of-features 8

## Alpha and beta diversity analysis: perch

### Filter samples

In [ ]:
!qiime feature-table filter-samples \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted.qza \
  --m-metadata-file data/metadata.tsv \
  --p-where "[Population]='Wild'" \
  --o-filtered-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_perch.qza

### Rarefaction analysis 

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_perch.qza \
  --i-phylogeny data/intermediate/qiime2/insertion_tree.qza \
  --p-max-depth 250788 \
  --p-steps 10 \
  --m-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/alpha_rarefaction_perch.qzv

### Generate core metric results

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny data/intermediate/qiime2/insertion_tree.qza \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_perch.qza \
  --m-metadata-file data/metadata.tsv \
  --p-sampling-depth 250788 \
  --output-dir data/intermediate/qiime2/core_metrics_results_perch

### Compare beta-diversity using robust Aitchison PCA 

In [ ]:
!qiime deicode rpca \
  --i-table data/intermediate/qiime2/table_merged_filtered_sepp_inserted_perch.qza \
  --p-min-feature-count 10 \
  --p-min-sample-count 1000 \
  --output-dir data/intermediate/qiime2/rpca_perch

!qiime emperor biplot \
  --i-biplot data/intermediate/qiime2/rpca_perch/biplot.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --m-feature-metadata-file data/intermediate/qiime2/taxonomy_silva132.qza \
  --o-visualization data/intermediate/qiime2/rpca_perch/biplot.qzv \
  --p-number-of-features 8